# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [1]:
import pandas as pd
import altair as alt
import regex as re

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [2]:
def legislators_df():
    df = pd.read_csv("data/legislators.csv")

    # post on how to use regex for pandas column:
    # https://stackoverflow.com/questions/22588316/pandas-applying-regex-to-replace-values 
    # handle DC and PR
    df["state"] = df["jurisdiction"].str.replace(r"^ocd-jurisdiction/country:us/district:dc/government", "DC", regex = True)
    df["state"] = df["state"].str.replace(r"ocd-jurisdiction/country:us/territory:pr/government", "PR", regex = True)

    # handle every other state
    df["state"] = df["state"].str.replace(r"ocd-jurisdiction/country:us/state:", "", regex = True)
    df["state"] = df["state"].str.replace(r"/government", "", regex = True)
    df["state"] = df["state"].str.upper()
    
    return df
    

# render dataframe
legislators_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,DC
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,DC
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,DC
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,DC
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,DC
...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,PR
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,PR
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,PR
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,PR


### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [3]:
def states_by_gender_initial(df):
    # get number of males / females in each state
    # https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby
    df_counts = df.groupby(["state", "gender"]).size().reset_index(name="count")

    # create chart
    # https://www.geeksforgeeks.org/stacked-bar-chart-with-selection-using-altair-in-python/
    # https://altair-viz.github.io/gallery/stacked_bar_chart.html

    # change y's label: https://github.com/vega/altair/issues/1465
    chart = alt.Chart(df_counts).mark_bar().encode( 
        x = "state", 
        y = alt.Y("count:Q", title = "Count of Records"),
        color = "gender") 

    return chart

# render chart
states_by_gender_initial(legislators_df())

alt.Chart(...)

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [4]:
def states_by_gender_improved(df):
    # get number of males / females in each state
    df_counts = df.groupby(["state", "gender"]).size().reset_index(name="count")
    
    # normalized bar chart: https://altair-viz.github.io/gallery/normalized_stacked_bar_chart.html
    chart = alt.Chart(df_counts).mark_bar().encode(
        x = "state",
        y = alt.Y("count:Q", title = "Count of Records").stack("normalize"),
        # change color: https://github.com/vega/altair/issues/1719 
        color = alt.Color("gender:N", scale = alt.Scale(domain=['Male', 'Female'], range = ["#1fc3aa", "#8624f5"]))
    )
    line = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule().encode(y='y')

    # https://github.com/vega/altair/issues/2059
    newchart = chart + line
    
    return newchart

# render chart
states_by_gender_improved(legislators_df())

alt.LayerChart(...)

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [5]:
def party_control_raw(df):
        # get number of males / females in each state
    df_counts = df.groupby(["state", "party"]).size().reset_index(name="count")
    
    # normalized bar chart: https://altair-viz.github.io/gallery/normalized_stacked_bar_chart.html
    chart = alt.Chart(df_counts).mark_bar().encode(
        x = "state",
        y = alt.Y("count:Q", title = "Count of Records").stack("normalize"),
        # change color: https://github.com/vega/altair/issues/1719 
        color = alt.Color("party:N")
    )
    line = alt.Chart(pd.DataFrame({"y": [0.5]})).mark_rule().encode(y = "y")

    # https://github.com/vega/altair/issues/2059
    newchart = chart + line
    
    return newchart

party_control_raw(legislators_df())

alt.LayerChart(...)

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [6]:
def clean_party_df():
    # start with the DataFrame from part 1 & return transformed copy
    df = legislators_df()

    # regex for replacing word if it contains a string
    # https://stackoverflow.com/questions/51398626/python-regular-expressions-replace-text-containing-a-word
    df["party_code"] = df["party"].str.replace(r".*\bDemocratic\b.*", "D", regex = True)
    df["party_code"] = df["party_code"].str.replace(r".*\bRepublican\b.*", "R", regex = True)
    
    # replace everything except for specific characters in regex
    # https://salesforce.stackexchange.com/questions/265746/how-do-i-replace-all-non-digit-characters-in-a-regex-except-for-the-period
    df["party_code"] = df["party_code"].str.replace(r'^(?![DR]$).+', "O", regex=True)

    return df

clean_party_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state,party_code
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,DC,O
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,DC,D
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,DC,D
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,DC,D
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,DC,D
...,...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,PR,O
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,PR,O
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,PR,O
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,PR,O


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [7]:
def party_control_by_chamber(df):
    df = clean_party_df()

    # separate by chamber
    df_lower = df[df["type"] == "lower"]
    df_upper = df[df["type"] == "upper"]

    # group by for the two chambers
    df_counts_lower = df_lower.groupby(["state", "party_code"]).size().reset_index(name="count")
    df_counts_upper = df_upper.groupby(["state", "party_code"]).size().reset_index(name="count")
    
    # create charts for the two houses
    chart_lower = alt.Chart(df_counts_lower).mark_bar().encode(
        x = alt.X("count:Q", title = "Count of Records").stack("normalize"),
        y = "state",
        # change color: https://github.com/vega/altair/issues/1719 
        color = alt.Color("party_code:N", scale = alt.Scale(domain = ["D", "R", "O"], range = ["#4169E1", "#880808", "#008000"]))
    ).properties(title = alt.Title("lower"))
    
    line = alt.Chart(pd.DataFrame({"x": [0.5]})).mark_rule().encode(x = "x")

    newchart_lower = chart_lower + line

    chart_upper = alt.Chart(df_counts_upper).mark_bar().encode(
        x = alt.X("count:Q", title = "Count of Records").stack("normalize"),
        y = "state",
        # change color: https://github.com/vega/altair/issues/1719 
        color = alt.Color("party_code:N", scale = alt.Scale(domain = ["D", "R", "O"], range = ["#4169E1", "#880808", "#008000"]))
    ).properties(title = alt.Title("upper"))
                 
    line = alt.Chart(pd.DataFrame({"x": [0.5]})).mark_rule().encode(x = "x")

    newchart_upper = chart_upper + line

    # facet the two charts
    # https://altair-viz.github.io/user_guide/compound_charts.html
    chart = alt.hconcat(newchart_lower, newchart_upper)

    return chart

party_control_by_chamber(clean_party_df())

alt.HConcatChart(...)

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [8]:
def population_combined_df():
    df = clean_party_df()
    # load population data
    population = pd.read_csv("data/populations.csv")
    population["state"] = population["state"].str.upper()

    data = df.groupby(["state", "type"]).size().reset_index(name="count")

    # separate by upper and lower chambers
    upper = data[data["type"] == "upper"]
    upper = upper.drop(columns = ["type"])
    upper = upper.rename(columns = {"count" : "upper"})
    
    lower = data[data["type"] == "lower"]
    lower = lower.drop(columns = ["type"])
    lower = lower.rename(columns = {"count" : "lower"})

    # join the three tables together
    pop_data = upper.merge(lower, on='state', how='inner').merge(population, on='state', how='inner')

    return pop_data


population_combined_df()

,state,upper,lower,pop_2020
0,AK,20,40,733374
1,AL,35,104,5024294
2,AR,35,98,3011490
3,AZ,30,59,7157902
4,CA,40,79,39538212
5,CO,35,65,5773707
6,CT,36,151,3605912
7,DE,21,41,989946
8,FL,40,119,21538216
9,GA,56,180,10713771


### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [9]:
def scatter_pop_size():
    df = population_combined_df()

    # create custom upper chart 
    # https://altair-viz.github.io/user_guide/marks/point.html#scatter-plot
    # https://altair-viz.github.io/user_guide/customization.html
    upper_chart = alt.Chart(df).mark_point(
        color = "purple",
        shape = "triangle-up"
    ).encode(
            x = "pop_2020:Q",
            y = "upper:Q"
        )

    lower_chart = alt.Chart(df).mark_point(
        color = "orange",
        shape = "triangle-down"
    ).encode(
            x = "pop_2020:Q",
            y = "lower:Q"
        )

    chart = upper_chart + lower_chart

    # change y axis label
    chart = chart.encode(
        y = alt.Y("upper,lower:Q", title = "Number of Seats"),
        x = alt.X("pop_2020:Q", title = "Population")
    
    )
    
    return chart

scatter_pop_size()

alt.LayerChart(...)

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [10]:
def scatter_pop_size_regression():
    df = population_combined_df()
    # get scatter plot for upper chamber
    upper_chart = alt.Chart(df).mark_point(
        color = "purple",
        shape = "triangle-up"
    ).encode(
            x = "pop_2020:Q",
            y = "upper:Q"
        )

    # create regression line
    # https://altair-viz.github.io/user_guide/transform/regression.html
    upper_regression = upper_chart + upper_chart.transform_regression("pop_2020", "upper").mark_line(color = "purple")

    # get scatter plot for lower chamber
    lower_chart = alt.Chart(df).mark_point(
        color = "orange",
        shape = "triangle-down"
    ).encode(
            x = "pop_2020:Q",
            y = "lower:Q"
        )

    # create regression line
    lower_regression = lower_chart + lower_chart.transform_regression("pop_2020", "lower").mark_line(color = "orange")

    # combine
    chart = upper_regression + lower_regression

    # change y axis label
    chart = chart.encode(
        y = alt.Y("upper,lower:Q", title = "Number of Seats"),
        x = alt.X("pop_2020:Q", title = "Population")
    
    )
    
    return chart
    
scatter_pop_size_regression()

alt.LayerChart(...)

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [11]:
def actions_df():
    df = pd.read_csv("data/actions_il-in-mi-wi_2021-2024.csv")

    df["date"] = pd.to_datetime(df["date"].str[:10])

    return df

### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [12]:
def actions_heatmap_scaled(df):

    df_agg = df.groupby(["state", "date"]).size().reset_index(name="count")

    # https://altair-viz.github.io/gallery/histogram_heatmap.html
    chart = alt.Chart(df_agg).mark_rect().encode(
        x = alt.X('yearweek(date):O', title='Week'),
        y = alt.Y('state:N', title = 'State'),
        color = alt.Color('count:Q', title='Action Count', scale=alt.Scale(scheme='greenblue'))
    ).properties(
        width=800,
        height=100
    )

    return chart

actions_heatmap_scaled(actions_df())

alt.Chart(...)

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [13]:
# MODIFY THIS LINE (exclude IL)
actions_heatmap_scaled(actions_df()[actions_df()["state"] != "IL"])

alt.Chart(...)

In [14]:
# MODIFY THIS LINE (only IL)
actions_heatmap_scaled(actions_df()[actions_df()["state"] == "IL"])

alt.Chart(...)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [15]:
def actions_cumulative(df):

    df_agg = df.groupby(["state", "date"]).size().reset_index(name="count")

    # https://github.com/vega/altair/discussions/2535
    chart = alt.Chart(df_agg).transform_window(
        cumulative='sum(count)',
        sort=[{"field": "date"}],
        groupby=['state'],
    ).mark_line().encode(
        x = 'date:T',
        y = 'cumulative:Q',
        color = "state:N"
    )

    return chart

actions_cumulative(actions_df())

alt.Chart(...)